In [1]:
import pymysql, os
import pandas as pd
from flask import Flask, request, render_template, redirect, url_for

# 00:read_only, 01:create_nano_user_only; 10:create_web_user_only; 11:gold
key_list = [os.urandom(20).hex() for i in range(4)]

connect = pymysql.connect(host='127.0.0.1', port=3306, \
    user='root', passwd='1234', db='user', charset='utf8')
cursor = connect.cursor()

app = Flask(__name__)

In [2]:
@app.route('/')
def root():
    return render_template('root.html', state='init')

@app.route('/login', methods=['POST'])
def login():
    cursor.execute("select* from web where username='{}' and password=MD5('{}');".format(\
            request.values['username'], request.values['password']))
    L = cursor.fetchall()
    if len(L):
        return redirect(url_for('menu', key=key_list[ L[0][2]*2+L[0][3] ]))
    return render_template('root.html', state='error')

@app.route('/menu_<key>')
def menu(key):
    return render_template('menu.html', key=key)

@app.route('/menu_<key>/create_web_user')
def create_web_user(key):
    if key == key_list[3]:
        return render_template('create_web_user.html', key=key, dis='F')
    if key == key_list[2]:
        return render_template('create_web_user.html', key=key, dis='T')
    return "Permission denied !!!"

@app.route('/menu_<key>/create_web_user/processing', methods=['POST'])
def create_web_user_process(key):
    username, password = request.values['username'], request.values['password']
    webuser, nanouser = 'webuser' in request.values, 'nanouser' in request.values
    try:
        cursor.execute("insert into web(username, password, webuser, nanouser) values(\
            '{}', MD5('{}'), {}, {});".format(username, password, webuser, nanouser))
        connect.commit()
        print("create web user success")
    except:
        print("create web user error! username already exist!")
        return "create web user error! username already exist!"
    return redirect(url_for('create_success', key=key, target='web'))

@app.route('/menu_<key>/create_<target>_user/success')
def create_success(key, target):
    return render_template('create_success.html', key=key, arg=target)

@app.route('/menu_<key>/create_nano_user')
def create_nano_user(key):
    if key == key_list[3] or key == key_list[1]:
        return render_template('create_nano_user.html', key=key)
    return "Permission denied !!!"

@app.route('/menu_<key>/create_nano_user/processing', methods=['POST'])
def create_nano_user_process(key):
    ID, group_no, account, password, IP = request.values['ID'], request.values['group_no'], \
        request.values['account'], request.values['password'], request.values['IP']
    gdb = "group{}".format(group_no)
    print(ID, group_no, account, password, gdb, IP)
    cursor.execute("show databases;")
    L = [ db[0] for db in cursor.fetchall() ]
    if gdb not in L:
        cursor.execute("create database {};".format(gdb))
    try:
        cursor.execute("insert into user.nano(ID, group_no, account, password, IP) values(\
            '{}', '{}', '{}', MD5('{}'), '{}');".format(ID, group_no, account, password, IP))
        cursor.execute("use {};".format(gdb))
        cursor.execute("create table a{}(data float, prediction float, CPU float, memory float, \
            time timestamp, notes char(20))".format(ID))
        connect.commit()
        print("create nano user success")
    except:
        print("create nano user error! ID already exist!")
        return "create nano user error! ID already exist!"
    return redirect(url_for('create_success', key=key, target='nano'))

@app.route('/menu_<key>/monitor')
def monitor(key):
    df = pd.read_sql("select ID,group_no,IP from user.nano;", connect)
    df['state'] = [ not os.system("ping -w 1000 {} -n 1".format(df['IP'][i])) \
        for i in range(len(df)) ]

    connections = {}
    for i in range(len(df)):
        connections[ df['ID'][i] ] = df['state'][i]

    df = df.groupby(['group_no'])['ID'].apply(list).reset_index(name='IDs')
    G = {}
    for i in range(len(df)):
        G[ df['group_no'][i] ] = df['IDs'][i]
    return render_template('monitor.html', key=key, connections=connections, G=G)

@app.route('/menu_<key>/monitor/<group_no>/<ID>')
def monitor_nano(key, group_no, ID):
    return render_template('monitor_nano.html', key=key, group_no=group_no, ID=ID)
    
@app.route('/menu_<key>/monitor/<group_no>/<ID>/update')
def monitor_nano_update(key, group_no, ID):  
    if 0:
        df = pd.read_sql("select* from group{}.a{} order by time desc limit 10".format(\
            group_no, ID), connect)
        return {'data':list(df['data']), 'prediction':list(df['prediction']), \
            'CPU':list(df['CPU']), 'memory':list(df['memory']), \
            'time':list(map(lambda x:str(x), df['time'])) }        
    else:
        from random import random
        return {'data':[round(random(),3) for i in range(10)], \
                'prediction':[round(random(),3) for i in range(10)], \
                'CPU':[round(random(),3) for i in range(10)], \
                'memory':[round(random(),3) for i in range(10)], \
                'time':[round(random(),3) for i in range(10)] }
        
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [3]:
# modify
# 1 create_nano_user_process: add IP column
# 2 monitor function
# 3 monitor nano function
# 4 mobitor nano update (partial loading)
# 5 menu.html: add monitor href

In [4]:
df = pd.read_sql("select* from group{}.a{} order by time desc limit 10".format(\
            0, '0000'), connect)
D = {'data':list(df['data']), 'prediction':list(df['prediction']), 'CPU':list(df['CPU']), \
    'memory':list(df['memory']), 'time':list(map(lambda x:str(x), df['time'])) }
D

{'data': [4.0, 2.0, 6.0, 1.0, 1.0, 2.0, 3.0, 1.0, 6.0, 2.0],
 'prediction': [0.0, 19.0, 19.0, 0.0, 4.0, 21.0, 25.0, 23.0, 27.0, 1.0],
 'CPU': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'memory': [40.9, 40.9, 40.9, 40.9, 40.9, 33.1, 33.1, 33.0, 33.1, 33.1],
 'time': ['2020-06-23 09:59:34',
  '2020-06-23 09:59:32',
  '2020-06-23 09:59:31',
  '2020-06-23 09:59:30',
  '2020-06-23 09:59:29',
  '2020-06-22 16:57:28',
  '2020-06-22 16:57:27',
  '2020-06-22 16:57:26',
  '2020-06-22 16:57:25',
  '2020-06-22 16:57:24']}

In [5]:
from random import random
str([random() for i in range(10)])

'[0.6061794743339328, 0.48125222405099266, 0.7588205603798269, 0.9453187523625525, 0.1522885413724494, 0.3134656920185711, 0.8241069857206776, 0.012033311346071995, 0.7076205401960911, 0.512353424208372]'

In [6]:
import datetime
str(datetime.datetime.now())

'2020-06-29 15:06:57.858876'